##Esse arquivo possui como objetivo fazer a quantificação dos dados de resíduos biosólidos de água e da produção de Biochar aplicando conhecimentos de análise e transformação de dados. Os dados presente foram extraídos das fontes especificadas no artigo de estudo e alocados no Drive


## Preparo de Ambiente

In [ ]:
#importar bibliotecas
import pandas as pd
import numpy as np


In [ ]:
# Conectar ao Drive para trabalhar com os dados nele presentes
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Análise exploratória, limpeza e filtragens na tabela de resíduos

In [ ]:
dfWater = pd.read_csv('/content/drive/MyDrive/Analise Lefebvre/Dados/Waste water biosolids.csv')

In [ ]:
dfWater.isna().sum()

ISO3                      0
Country                   0
Residence Type            0
Service Type              0
Year                      0
Coverage                  0
Population                0
Safely managed element    0
dtype: int64

In [ ]:
dfWater['Country'].unique().shape

(165,)

In [ ]:
dfWater = dfWater[dfWater['Safely managed element'] == 'Sewage treated']

In [ ]:
dfrt = dfWater[dfWater['Residence Type'] == 'total']


In [ ]:
dfrt

,ISO3,Country,Residence Type,Service Type,Year,Coverage,Population,Safely managed element
0,ABW,Aruba,total,Sanitation,2020,4.67210,4.979760e+03,Sewage treated
5,ALB,Albania,total,Sanitation,2020,26.41708,7.573378e+05,Sewage treated
14,AND,Andorra,total,Sanitation,2020,100.00000,7.770000e+04,Sewage treated
23,ARE,United Arab Emirates,total,Sanitation,2020,92.54676,8.595085e+06,Sewage treated
33,ARM,Armenia,total,Sanitation,2020,0.00000,0.000000e+00,Sewage treated
...,...,...,...,...,...,...,...,...
1052,VUT,Vanuatu,total,Sanitation,2020,0.00000,0.000000e+00,Sewage treated
1061,WSM,Samoa,total,Sanitation,2020,0.00000,0.000000e+00,Sewage treated
1068,YEM,Yemen,total,Sanitation,2020,20.84273,6.728877e+06,Sewage treated
1075,ZAF,South Africa,total,Sanitation,2020,46.86356,2.755667e+07,Sewage treated


## Análise exploratória, limpeza e filtragens na tabela de especificações do resíduo

In [ ]:
dfWaterRatios = pd.read_excel('/content/drive/MyDrive/Analise Lefebvre/Dados/tabela 5 dados suplementares 1.xlsx')

In [ ]:
dfWaterRatios.columns

Index(['Biosolids production\n(DM basis)\n[kg yr-1 person-1]',
       'Collection factor\n\n\n[%]', 'Biochar yield\n(DM basis)\n\n[%]',
       'Biochar organic carbon content (DM basis)\n\n[%]'],
      dtype='object')

In [ ]:
dfWaterRatios[['Collection factor\n\n\n[%]', 'Biochar yield\n(DM basis)\n\n[%]',
       'Biochar organic carbon content (DM basis)\n\n[%]']] /= 100

In [ ]:
dfWaterRatios = dfWaterRatios.rename(columns = {'Biosolids production\n(DM basis)\n[kg yr-1 person-1]': 'Biosolids production (DM Basis) kg yr-1 person-1',
       'Collection factor\n\n\n[%]': 'Collection factor', 'Biochar yield\n(DM basis)\n\n[%]': 'Biochar yield (DM basis)',
       'Biochar organic carbon content (DM basis)\n\n[%]': 'Biochar organic carbon content (DM basis)'})

In [ ]:
dfWaterRatios

,Biosolids production (DM Basis) kg yr-1 person-1,Collection factor,Biochar yield (DM basis),Biochar organic carbon content (DM basis)
0,25.6,1.0,0.552,0.38


## Realizar cálculos e filtrar os resultados dos resíduos e da produção de biochar

In [ ]:

# Inicializar uma lista para armazenar os resultados
resultados = []
linhaRatios = dfWaterRatios.iloc[0]

# Iterar sobre os países
for pais in dfWater['Country'].unique():
    # Filtrar as linhas correspondentes ao país atual
    df_producao_pais = dfWater[dfWater['Country'] == pais]

    # Inicializar a soma dos resíduos para o país atual
    soma_residuos_pais = 0
    soma_biochar = 0

    # Iterar sobre as culturas do país atual
    for index, row in df_producao_pais.iterrows():

      # Somar os resíduos considerando os coeficientes correspondentes do tipo de resíduo
      soma_biochar += row['Population'] * linhaRatios['Biosolids production (DM Basis) kg yr-1 person-1'] * linhaRatios['Collection factor'] * linhaRatios['Biochar yield (DM basis)'] #* linhaRatios['Biochar organic carbon content (DM basis)']
      soma_residuos_pais += row['Population'] * linhaRatios['Biosolids production (DM Basis) kg yr-1 person-1'] * linhaRatios['Collection factor'] #* linhaRatios['Biochar organic carbon content (DM basis)']


    # Armazenar o resultado para o país atual na lista
    resultados.append({'País': pais, 'Soma de Resíduos': soma_residuos_pais,  'Biochar Produzido': soma_biochar})

# Criar um DataFrame a partir da lista de resultados
df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados[['Soma de Resíduos', 'Biochar Produzido']] /= 1000

In [ ]:
df_resultados

,País,Soma de Resíduos,Biochar Produzido
0,Aruba,1.274819e+02,70.369991
1,Albania,3.877570e+04,21404.183878
2,Andorra,3.978240e+03,2195.988501
3,United Arab Emirates,4.400684e+05,242917.734875
4,Argentina,3.145091e+05,173609.012706
...,...,...,...
160,Samoa,0.000000e+00,0.000000
161,Yemen,3.445185e+05,190174.197269
162,South Africa,1.410902e+06,778817.730790
163,Zambia,3.926971e+02,216.768787


In [ ]:
countries = [
    'Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia',
    'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize',
    'Benin', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
    'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
    'Canada', 'Chad', 'Chile', 'China', 'Colombia', 'Democratic Republic of the Congo',
    'Congo', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Czechia', 'Denmark',
    'Djibouti', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea',
    'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Finland', 'France', 'Gabon', 'Gambia',
    'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana',
    'Haiti', 'Honduras', 'Hungary', 'India', 'Indonesia', 'Iran (Islamic Republic of)', 'Iraq',
    'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
    "Democratic People's Republic of Korea", 'Republic of Korea', 'Kuwait', 'Kyrgyzstan',
    "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya',
    'Lithuania', 'Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Mauritania', 'Mexico',
    'Republic of Moldova', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar',
    'Namibia', 'Nepal', 'Netherlands (Kingdom of the)', 'New Zealand', 'Nicaragua', 'Niger',
    'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Paraguay', 'Peru',
    'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda',
    'Saudi Arabia', 'Senegal', 'Serbia', 'Sierra Leone', 'Slovakia', 'Slovenia', 'Somalia',
    'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Sweden', 'Switzerland',
    'Syrian Arab Republic', 'Tajikistan', 'United Republic of Tanzania', 'Thailand', 'Timor-Leste',
    'Togo', 'Trinidad and Tobago', 'Tunisia', 'Türkiye', 'Turkmenistan', 'Uganda', 'Ukraine',
    'United Arab Emirates', 'United Kingdom of Great Britain and Northern Ireland',
    'United States of America', 'Uruguay', 'Uzbekistan', 'Venezuela (Bolivarian Republic of)',
    'Viet Nam', 'Yemen', 'Zambia', 'Zimbabwe'
]



In [ ]:
df_filtrado = df_resultados[df_resultados['País'].isin(countries)]

In [ ]:
df_filtrado.sort_values(by='País')
df_filtrado

,País,Soma de Resíduos,Biochar Produzido
1,Albania,3.877570e+04,21404.183878
3,United Arab Emirates,4.400684e+05,242917.734875
4,Argentina,3.145091e+05,173609.012706
5,Armenia,0.000000e+00,0.000000
7,Australia,6.079549e+05,335591.120129
...,...,...,...
158,Viet Nam,7.715601e+04,42590.119941
161,Yemen,3.445185e+05,190174.197269
162,South Africa,1.410902e+06,778817.730790
163,Zambia,3.926971e+02,216.768787


In [ ]:
#salvar resultado em excel
df_filtrado.to_excel("/content/drive/MyDrive/Analise Lefebvre/Dados quantificados/residues water.xlsx", index=True)

In [ ]:
len(df_filtrado)


130